In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
#import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [2]:
from funcoes import extract_stats

# Tentativa de com base equilibrada p%

In [3]:
p = 30

In [4]:
combinacoes_equilibradas = pd.read_csv("../combinacoes"+str(p)+".txt", header = 0, sep=",")

In [5]:
# le = preprocessing.LabelEncoder()
# nomes = []
# for i in sorted(glob('../letras/caracter*.png')):
#     nomes.append(i.split("/")[1] + "/" + i.split("/")[2])

In [6]:
# le.fit(nomes)

In [7]:
# combinacoes30["char1"] = le.transform(combinacoes30["char1"])
# combinacoes30["char2"] = le.transform(combinacoes30["char2"])

In [8]:
#X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["char1", "char2", "MSE", "ISS", "MSE_centro", "ISS_centro"]],
#                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes_equilibradas[["MSE", "ISS", "MSE_centro", "ISS_centro",\
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",\
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]],
                                                                    combinacoes_equilibradas["resposta"], test_size=0.3, random_state=0)

## SVM

In [10]:
%%time
#from sklearn.svm import SVC
#clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

#SVMparams = { 'kernel' : ('linear', 'poly', 'rbf')} #, 'sigmoid')}

#svr = SVC()
#clf = GridSearchCV(svr, SVMparams)

#clf.fit(X_train, y_train)
#print( "Score: {0}".format(clf.score(X_test, y_test)) )
#print( "Best parameters: " + clf.best_params_ )

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


## Decision Tree

In [11]:
%%time
from sklearn import tree

DTparams = { 'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 3, 7, 40)}

svr = tree.DecisionTreeClassifier()
clf1 = GridSearchCV(svr, DTparams) #criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clf1.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clf1.score(X_test, y_test)) )
print( "Best parameters: " + str(clf1.best_params_) )

Score: 0.7288888888888889
Best parameters: {'criterion': 'gini', 'min_samples_leaf': 40, 'min_samples_split': 40}
CPU times: user 16.9 s, sys: 40 ms, total: 16.9 s
Wall time: 17.1 s


# Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [14]:
%%time

RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.7485470085470085
Best parameters: {'criterion': 'gini', 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 10}
CPU times: user 1min 24s, sys: 928 ms, total: 1min 24s
Wall time: 1min 25s


In [15]:
%%time

from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier(criterion = 'entropy', n_estimators=10, min_samples_leaf=40,
                                       min_samples_split=5)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )

Score: 0.7516239316239316
CPU times: user 540 ms, sys: 8 ms, total: 548 ms
Wall time: 547 ms


## Logistic Regression

In [16]:
%%time
from sklearn.linear_model import LogisticRegression

LRparams = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag')}

svr = LogisticRegression()
clfLR = GridSearchCV(svr, LRparams)
clfLR.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfLR.score(X_test, y_test)) )
print( "Best parameters: " + str(clfLR.best_params_) )

/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The li

Score: 0.7367521367521368
Best parameters: {'solver': 'newton-cg'}
CPU times: user 16.3 s, sys: 31.4 s, total: 47.6 s
Wall time: 16 s


/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


# Multi-layer Perceptron classifier

In [17]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
            'activation' : ('identity', 'logistic', 'tanh', 'relu'),
            'solver' : ('lbfgs', 'sgd', 'adam')}

svr = MLPClassifier()
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.7046153846153846
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 1.81 s, sys: 3.75 s, total: 5.56 s
Wall time: 1.82 s


# Tentativa com base de p = 50%

In [18]:
p = 50

In [19]:
combinacoes_equilibradas = pd.read_csv("../combinacoes"+str(p)+".txt", header = 0, sep=",")

In [24]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes_equilibradas[["MSE", "ISS", "MSE_centro", "ISS_centro",\
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",\
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]],
                                                                    combinacoes_equilibradas["resposta"], test_size=0.3, random_state=0)

## SVM

In [25]:
%%time
#from sklearn.svm import SVC
#clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

#SVMparams = { 'kernel' : ('linear', 'poly', 'rbf')} #, 'sigmoid')}

#svr = SVC()
#clf = GridSearchCV(svr, SVMparams)

#clf.fit(X_train, y_train)
#print( "Score: {0}".format(clf.score(X_test, y_test)) )
#print( "Best parameters: " + clf.best_params_ )

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


## Decision Tree

In [26]:
%%time
from sklearn import tree

DTparams = { 'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 3, 7, 40)}

svr = tree.DecisionTreeClassifier()
clf1 = GridSearchCV(svr, DTparams) #criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clf1.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clf1.score(X_test, y_test)) )
print( "Best parameters: " + str(clf1.best_params_) )

Score: 0.6381766381766382
Best parameters: {'criterion': 'gini', 'min_samples_leaf': 40, 'min_samples_split': 10}
CPU times: user 9.81 s, sys: 132 ms, total: 9.94 s
Wall time: 9.95 s


# Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [29]:
%%time

RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.6717948717948717
Best parameters: {'criterion': 'gini', 'min_samples_leaf': 5, 'min_samples_split': 40, 'n_estimators': 10}
CPU times: user 50.8 s, sys: 972 ms, total: 51.8 s
Wall time: 51.8 s


In [33]:
%%time

from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier(criterion = 'gini', n_estimators=10, min_samples_leaf=5,
                                       min_samples_split=40)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )

Score: 0.6715099715099715
CPU times: user 596 ms, sys: 4 ms, total: 600 ms
Wall time: 604 ms


## Logistic Regression

In [31]:
%%time
from sklearn.linear_model import LogisticRegression

LRparams = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag')}

svr = LogisticRegression()
clfLR = GridSearchCV(svr, LRparams)
clfLR.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfLR.score(X_test, y_test)) )
print( "Best parameters: " + str(clfLR.best_params_) )

/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearc

Score: 0.5911680911680912
Best parameters: {'solver': 'newton-cg'}
CPU times: user 7.78 s, sys: 15.1 s, total: 22.9 s
Wall time: 6.94 s


# Multi-layer Perceptron classifier

In [32]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
            'activation' : ('identity', 'logistic', 'tanh', 'relu'),
            'solver' : ('lbfgs', 'sgd', 'adam')}

svr = MLPClassifier()
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.49173789173789173
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 620 ms, sys: 1.19 s, total: 1.81 s
Wall time: 459 ms


# Binarizando o melhor modelo

In [73]:
#imgA = skio.imread("../letras/caracter1.png")
#imgB = skio.imread("../letras/caracter15.png")

#mse, iss, mse_centro, iss_centro,\
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,\
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro = extract_stats(imgA, imgB)
        
#print(clfRF_balanced.predict( [[mse, iss, mse_centro, iss_centro,
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro]]))

In [34]:
joblib.dump(clfRF_balanced, 'classifier_balanced.pkl') 

['classifier_balanced.pkl']